In [1]:
# 🔧 Data Handling
import pandas as pd
import numpy as np

# 📅 Date & Time
from datetime import datetime

import time


In [2]:
!pip install pycountry

  Using cached pycountry-24.6.1-py3-none-any.whl.metadata (12 kB)
Using cached pycountry-24.6.1-py3-none-any.whl (6.3 MB)


In [3]:
# ☁️ AWS S3 Interaction
import boto3
from io import StringIO

# 🌍 Country Info (used for dimregion country column)
import pycountry

# 🧼 Optional: Logging if you want to trace/debug stuff
import logging

In [ ]:
aws_access_key = '<your_aws_access_key>'
aws_secret_key = '<your_own_secret>' 
aws_region = 'us-east-2'
s3_bucket_name = 'covidproject-buc'


athena_client = boto3.client(
                "athena",
                aws_access_key_id = aws_access_key,
                aws_secret_access_key = aws_secret_key,
                region_name = aws_region
)
# ....... CODE TO EXECUTE A SIMPLE QUERY USING ATHENA JUST TO VIEW THE DATA, NOT NECCESSARY

In [5]:

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    aws_access_key_id=aws_access_key,
    aws_secret_access_key=aws_secret_key,
    region_name=aws_region
)

def get_pd_frame_data(file_info):
    file_key = s3_client.get_object(Bucket=s3_bucket_name, Key=file_info['file_path'])
    csv_data = file_key['Body'].read().decode('utf-8')  # Decode the byte data into string
    file_info['data'] = pd.read_csv(StringIO(csv_data)) 

data_entities = [{'df_name': 'hospital_beds', 'file_path': 'covidproject_tables/hospital_beds/rearc_usa_hospital_beds.csv', 'data':None},
                 {'df_name': 'covid_data', 'file_path': 'covidproject_tables/covid-data/covid_testing_data_states_daily.csv', 'data':None},
                {'df_name': 'enigma_jhud', 'file_path': 'covidproject_tables/enigma-jhud/enigma_jhud.csv', 'data':None}]

for i in range(len(data_entities)):
    print("added data for : ", data_entities[i]['df_name'])
    get_pd_frame_data(data_entities[i])

hospital_beds = data_entities[0]['data']
covid_data = data_entities[1]['data']
enigma_jhud = data_entities[2]['data']

        
# GET hospital_beds data and store it in a python dataframe
# s3_hospitalsData_key = 
# response_hospitals = s3_client.get_object(Bucket=s3_bucket_name, Key=s3_hospitalsData_key)

# # Read the CSV content into a pandas DataFrame
# csv_data = response_hospitals['Body'].read().decode('utf-8')  # Decode the byte data into string
# hospital_beds = pd.read_csv(StringIO(csv_data))  # Load into DataFrame

# Verify the DataFrame

# lets create the hospital dimension table 
dimhospital = hospital_beds[['FIPS', 'HOSPITAL_NAME', 'HOSPITAL_TYPE', 'HQ_ADDRESS', 'HQ_ADDRESS1', 'HQ_CITY', 'HQ_STATE', 'COUNTY_NAME', 'latitude', 'longtitude']]

print("dimhospital ready")
##############################################################################
# for the facts table
factscovid_daily = covid_data[['fips', 'date', 'state', 'positive', 'negative', 'pending', 'hospitalized', 'dateModified', 'recovered', 'deathConfirmed', 'hospitalizedDischarged']]
factscovid_enigma = enigma_jhud[['fips', 'active', 'recovered', 'confirmed']]
factscovid = pd.merge(factscovid_daily, factscovid_enigma, on = 'fips', how = 'inner')

print("factscovid ready")



#############################################################################
#for the region dimension table 
dimregion = enigma_jhud[['fips', 'province_state', 'country_region', 'latitude', 'longitude']]

print("dimregion ready")
print(len(dimregion))

def get_country_name(state_name):
    if pd.isna(state_name):
        return None  # Return None or any default value you prefer for NaN entries
    state_name = str(state_name)  # Convert the state_name to a string
    for subdivision in pycountry.subdivisions:
        if subdivision.name.lower() == state_name.lower():
            return pycountry.countries.get(alpha_2=subdivision.country_code).name
    return None
dimregion['country'] = dimregion['province_state'].apply(get_country_name)

print("country name has been added")
#############################################################################
# lastly, lets work on the dimension date table 
#first, I covert the date column to datetime format
covid_data['date'] = pd.to_datetime(covid_data['date'])
# now, lets get the month, year, weekend and day 
dimdate = pd.DataFrame({
    'date': covid_data['date'],
    'day': covid_data['date'].dt.day,
    'month': covid_data['date'].dt.month,
    'year': covid_data['date'].dt.year,
    'is_weekend': covid_data['date'].dt.weekday >= 5
})

added data for :  hospital_beds
added data for :  covid_data
added data for :  enigma_jhud
dimhospital ready
factscovid ready
dimregion ready
222804
country name has been added


/tmp/ipykernel_118/1746393962.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dimregion['country'] = dimregion['province_state'].apply(get_country_name)


In [6]:
bucket_name = 'covidproject-dimdata'
s3_resource = boto3.resource('s3')

# Dimregion back to S3 bucket in Output directory
csv_buffer = StringIO()
dimregion.to_csv(csv_buffer, index=False)
s3_resource.Object(bucket_name, 'output/dimregion.csv').put(Body=csv_buffer.getvalue())

# Dimdate back to S3 bucket in Output directory
csv_buffer = StringIO()
dimdate.to_csv(csv_buffer, index=False)
s3_resource.Object(bucket_name, 'output/dimdate.csv').put(Body=csv_buffer.getvalue())

# Dimregion back to S3 bucket in Output directory
csv_buffer = StringIO()
dimhospital.to_csv(csv_buffer, index=False)
s3_resource.Object(bucket_name, 'output/dimhospital.csv').put(Body=csv_buffer.getvalue())

# Factscovid back to S3 bucket in Output directory
csv_buffer = StringIO()
factscovid.to_csv(csv_buffer, index=False)
s3_resource.Object(bucket_name, 'output/factscovid.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'H8K1D6AJD5FSWSKB',
  'HostId': 'mYcHSz3u88KSHmX0eIepKckyMIM4QBUnrMDwBvRADVKBTTnfTO+FCy/kj6G7DDsDIGCn4YIk171r1VfSnIdILAVHHacBWT9Z',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'mYcHSz3u88KSHmX0eIepKckyMIM4QBUnrMDwBvRADVKBTTnfTO+FCy/kj6G7DDsDIGCn4YIk171r1VfSnIdILAVHHacBWT9Z',
   'x-amz-request-id': 'H8K1D6AJD5FSWSKB',
   'date': 'Wed, 16 Apr 2025 14:40:09 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"857d92ec07d1ec323198076737b15e00"',
   'x-amz-checksum-crc32': 'vBS1nA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"857d92ec07d1ec323198076737b15e00"',
 'ChecksumCRC32': 'vBS1nA==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}